# Example: Validate Model Outputs for correctness

For safety-critical applications we can not accept deviations at the model ouputs caused by the deployment method. The following shows how to verify if the generated model outputs are as expected.

*Warning:* The current version can only verify the bit-excactness of model outputs. Hence why it is very sensitive to even small derivations compared to the reference (golden) outputs. This limitation might be eliminated with a future revision of MLonMCUs `validate` feature.

## Supported components

**Models:** Any (`aww` and `resnet` used below)

**Frontends:** Any (`tflite` used below)

**Frameworks/Backends:** Any (`tvmaotplus` and `tflmi` used below)

**Platforms/Targets:** Any target supported by `mlif` or `espidf` platform

**Features:** `validate` and `debug` platform features have to be enabled 

## Prerequisites

Set up MLonmCU as usual, i.e. initialize an environment and install all required dependencies. Feel free to use the following minimal `environment.yml.j2` template:

```yaml
---
home: "{{ home_dir }}"
logging:
  level: DEBUG
  to_file: false
  rotate: false
cleanup:
  auto: true
  keep: 10
paths:
  deps: deps
  logs: logs
  results: results
  plugins: plugins
  temp: temp
  models:
    - "{{ home_dir }}/models"
    - "{{ config_dir }}/models"
repos:
  tensorflow:
    url: "https://github.com/tensorflow/tflite-micro.git"
    ref: f050eec7e32a0895f7658db21a4bdbd0975087a5
  tvm:
    url: "https://github.com/apache/tvm.git"
    ref: de6d8067754d746d88262c530b5241b5577b9aae
  etiss:
    url: "https://github.com/tum-ei-eda/etiss.git"
    ref: 4d2d26fb1fdb17e1da3a397c35d6f8877bf3ceab
  mlif:
    url: "https://github.com/tum-ei-eda/mlonmcu-sw.git"
    ref: 4b9a32659f7c5340e8de26a0b8c4135ca67d64ac
frameworks:
  default: tvm
  tflm:
    enabled: true
    backends:
      default: tflmi
      tflmi:
        enabled: true
        features: []
    features: []
  tvm:
    enabled: true
    backends:
      default: tvmaotplus
      tvmaotplus:
        enabled: true
        features: []
    features: []
frontends:
  tflite:
    enabled: true
    features: []
toolchains:
  gcc: true
platforms:
  mlif:
    enabled: true
    features:
      debug: true
      validate: true
targets:
  default: etiss_pulpino
  etiss_pulpino:
    enabled: true
    features: []
```

Do not forget to set your `MLONMCU_HOME` environment variable first if not using the default location!

## Usage

*Hint*: Due to the program being build in debug mode and running one inference for each provided input-output combination, the simulation time will likely decrease by some factors. Add the `--parallel` flag to your command line to allow MLonMCU to run multiple simulations in parallel.

*Hint:* We are not able to provide reference data for every model in out model zoo. If you might want to add reference data for your own models, see: TODO

### A) Command Line Interface

As an example, let's see if the `tflmi` and `tvmaotplus` backend produce different model outputs for the same model.

To enable the validation, just add `--feature debug --feature validate` to the command line:

In [2]:
!python -m mlonmcu.cli.main flow run aww resnet -b tflmi -b tvmaotplus -t etiss_pulpino -f debug -f validate

INFO - Loading environment cache from file
INFO - Successfully initialized cache
INFO - Loading extensions.py (User)
INFO - [session-281]  Processing stage LOAD
INFO - [session-281]  Processing stage BUILD
INFO - [session-281]  Processing stage COMPILE
INFO - [session-281]  Processing stage RUN
ERROR - A platform error occured during the simulation. Reason: OUTPUT_MISSMATCH
WARNING - unexpected script output (cycles)
INFO - All runs completed successfuly!
INFO - Postprocessing session report
INFO - [session-281] Done processing runs
INFO - Report:
   Session  Run   Model Frontend Framework     Backend Platform         Target      Cycles  MIPS  Total ROM  Total RAM  ROM read-only  ROM code  ROM misc  RAM data  RAM zero-init data  Validation           Features                                             Config Postprocesses Comment
0      281    0     aww   tflite      tflm       tflmi     mlif  etiss_pulpino   708303725    72     372308      35989         105528    266632       148     

Since we are building in debug mode, most of the reported metrics are not meaningful. Let's get rid of the using the `filter_cols` postprocess:

In [3]:
!python -m mlonmcu.cli.main flow run aww resnet -b tflmi -b tvmaotplus -t etiss_pulpino -f debug -f validate \
        --postprocess filter_cols --config filter_cols.keep="Model,Backend,Validation"

INFO - Loading environment cache from file
INFO - Successfully initialized cache
INFO - Loading extensions.py (User)
INFO - [session-282]  Processing stage LOAD
INFO - [session-282]  Processing stage BUILD
INFO - [session-282]  Processing stage COMPILE
INFO - [session-282]  Processing stage RUN
ERROR - A platform error occured during the simulation. Reason: OUTPUT_MISSMATCH
WARNING - unexpected script output (cycles)
INFO - [session-282]  Processing stage POSTPROCESS
INFO - All runs completed successfuly!
INFO - Postprocessing session report
INFO - [session-282] Done processing runs
INFO - Report:
    Model     Backend  Validation
0     aww       tflmi        True
1     aww  tvmaotplus        True
2  resnet       tflmi        True
3  resnet  tvmaotplus       False


By investigating the 'Validation' column or the `OUTPUT_MISSMATCH` printed earlier (at least at the time of testing this example) you can see, that one out of 4 validation have failed. TVM beeing not bit-accurate for quantized models is a known issue which needs further investigation.

It is also possible to find out which model output has caused the missmatch by looking at the simulation outputs:

In [4]:
!python -m mlonmcu.cli.main flow run resnet -b tvmaotplus -t etiss_pulpino -f debug -f validate \
        --config etiss_pulpino.print_outputs=1

INFO - Loading environment cache from file
INFO - Successfully initialized cache
INFO - Loading extensions.py (User)
INFO - [session-283]  Processing stage LOAD
INFO - [session-283]  Processing stage BUILD
INFO - [session-283]  Processing stage COMPILE
INFO - [session-283]  Processing stage RUN
=== Setting up configurations ===
Initializer::loadIni(): Ini sucessfully loaded /var/tmp/ga87puy/mlonmcu/mlonmcu/workspace/deps/install/etiss/examples/base.ini
Initializer::loadIni(): Ini sucessfully loaded /tmp/etiss_dynamic_S8YtAiov0d.ini
Initializer::loadIni(): Ini sucessfully loaded /tmp/tmp5r2o0p5x/custom.ini
  Load Configs from .ini files:
ETISS: Info: Created new config container: global
ETISS: Info:   [BoolConfigurations]
ETISS: Info:     arch.or1k.ignore_sr_iee=false,
ETISS: Info:     etiss.enable_dmi=true,
ETISS: Info:     etiss.load_integrated_libraries=true,
ETISS: Info:     etiss.log_pc=false,
ETISS: Info:     jit.debug=false,
ETISS: Info:     jit.gcc.cleanup=true,
ETISS: Info:    